# Regional concentration index

The regional concentration index (RCI) measures the extent to which a collection of countries (a “region”) is trading among themselves relative to the rest of the world. It is computed as a ratio of two fractions: the share of the region’s trade with itself and the share of the region in world trade: 

$$
{RCI}_q = \frac{{Exports}_{qq} + {Imports}_{qq}}{{Exports}_q + {Imports}_q} \Bigg/ \frac{{Exports}_q + {Imports}_q}{{Exports}_{world} + {Imports}_{world}}
$$

The results of this notebook are saved as `flows.parquet` and `rci.parquet` in `data/`.

In [51]:
import numpy as np
import pandas as pd
import os
import re
import duckdb
from functions import asvector

## Set up

In [52]:
inputfolder = 'ADB-MRIO'
output1 = 'flows'
output2 = 'rci'
ta = 'ta'
index = 'mrio'

# inputfolder = 'ADB-MRIO62'
# output1 = 'flows62'
# output2 = 'rci62'
# ta = 'ta62'
# index = 'mrio62'

filelist = [file for file in os.listdir(f'../data/mrio/{inputfolder}') if not file.startswith('.')]
filelist.sort()

countries = pd.read_excel('../data/raw/countries.xlsx')
sectors = pd.read_excel('../data/raw/sectors.xlsx')
sectors = sectors.drop_duplicates(subset='ind', ignore_index=True)

G = 73      # Number of countries + ROW
# G = 63
N = 35      # Number of sectors
f = 5       # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

## Compute flows

In [53]:
DF = pd.DataFrame()

for file in filelist:
    
    year = re.search('[0-9]{4}', file).group()

    mrio = duckdb.sql(
        f"""
        SELECT * EXCLUDE(C0)
        FROM read_parquet('../data/mrio/{inputfolder}/{file}')
        """
    ).df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    v = np.where(x != 0, va/x, 0)
    Dx = np.diag(np.where(x != 0, 1/x, 0))
    A = Z @ Dx
    B = np.linalg.inv(np.eye(G*N) - A)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    VBY = np.diag(v) @ B @ Y

    DFi = pd.DataFrame({
        't': year,
        's': np.tile(np.arange(1, G+1).repeat(N), G),
        'r': np.arange(1, G+1).repeat(G*N),
        'i': np.tile(sectors['ind'], G*G),
        'i5': np.tile(sectors['ind5'], G*G),
        'i15': np.tile(sectors['ind15'], G*G),
        'flows': asvector(VBY)
    })
    DF = pd.concat([DF, DFi], ignore_index=True)

    print(f'{year} done')

DF.sort_values(['t', 's', 'r', 'i'], inplace=True)
DF.to_parquet(f'../data/{output1}.parquet', index=False)

2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


In [54]:
DF

,t,s,r,i,i5,i15,flows
0,2017,1,1,1,1,1,24580.928164
1,2017,1,1,2,1,2,17033.117412
2,2017,1,1,3,2,3,14724.957952
3,2017,1,1,4,2,3,1059.935367
4,2017,1,1,5,2,3,142.991659
...,...,...,...,...,...,...,...
1119085,2022,73,73,31,5,13,336582.229471
1119086,2022,73,73,32,5,14,245745.094921
1119087,2022,73,73,33,5,14,154924.793812
1119088,2022,73,73,34,5,15,129306.912080


## Compute RCI

In [55]:
grossexports = duckdb.sql(
    f"""
    SELECT t, s, r, Exports
    FROM read_parquet('../data/trade-accounting/{ta}.parquet')
    ORDER BY t, s, r
    """
).df()

flows = duckdb.sql(
    f"""
    SELECT t, s, r, SUM(flows) AS flows
    FROM read_parquet('../data/{output1}.parquet')
    WHERE s <> r
    GROUP BY t, s, r
    ORDER BY t, s, r
    """
).df()

In [56]:
rta = countries[[f'{index}', 'rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta']]

In [57]:
world_gross = grossexports.groupby(['t'])['Exports'].sum().reset_index()
world_gross.rename(columns={'Exports': 'world'}, inplace=True)

world_flows = flows.groupby(['t'])['flows'].sum().reset_index()
world_flows.rename(columns={'flows': 'world'}, inplace=True)

In [58]:
exports_gross = grossexports.merge(rta, left_on='s', right_on=f'{index}').drop([f'{index}'], axis=1)
exports_gross = exports_gross.melt(
    id_vars=['t', 's', 'Exports'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='rta',
    value_name='member'
)
exports_gross['rta'] = exports_gross['rta'].str.split('_').str[-1]
exports_gross.dropna(inplace=True)
exports_gross = exports_gross.groupby(['t', 'rta'])['Exports'].sum().reset_index()
exports_gross.rename(columns={'Exports': 'exports'}, inplace=True)

exports_flows = flows.merge(rta, left_on='s', right_on=f'{index}').drop([f'{index}'], axis=1)
exports_flows = exports_flows.melt(
    id_vars=['t', 's', 'flows'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='rta',
    value_name='member'
)
exports_flows['rta'] = exports_flows['rta'].str.split('_').str[-1]
exports_flows.dropna(inplace=True)
exports_flows = exports_flows.groupby(['t', 'rta'])['flows'].sum().reset_index()
exports_flows.rename(columns={'flows': 'exports'}, inplace=True)

In [59]:
imports_gross = grossexports.merge(rta, left_on='r', right_on=f'{index}').drop([f'{index}'], axis=1)
imports_gross = imports_gross.melt(
    id_vars=['t', 'r', 'Exports'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='rta',
    value_name='member'
)
imports_gross['rta'] = imports_gross['rta'].str.split('_').str[-1]
imports_gross.dropna(inplace=True)
imports_gross = imports_gross.groupby(['t', 'rta'])['Exports'].sum().reset_index()
imports_gross.rename(columns={'Exports': 'imports'}, inplace=True)

imports_flows = flows.merge(rta, left_on='r', right_on=f'{index}').drop([f'{index}'], axis=1)
imports_flows = imports_flows.melt(
    id_vars=['t', 'r', 'flows'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='rta',
    value_name='member'
)
imports_flows['rta'] = imports_flows['rta'].str.split('_').str[-1]
imports_flows.dropna(inplace=True)
imports_flows = imports_flows.groupby(['t', 'rta'])['flows'].sum().reset_index()
imports_flows.rename(columns={'flows': 'imports'}, inplace=True)

In [60]:
# Gross exports

within_gross = grossexports.merge(rta, left_on='s', right_on=f'{index}').drop([f'{index}'], axis=1)
within_gross = within_gross.melt(
    id_vars=['t', 's', 'r', 'Exports'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='exporter',
    value_name='member'
)
within_gross['exporter'] = within_gross['exporter'].str.split('_').str[-1]
within_gross.dropna(inplace=True)
within_gross.drop(['member'], axis=1, inplace=True)

within_gross = within_gross.merge(rta, left_on='r', right_on=f'{index}').drop([f'{index}'], axis=1)
within_gross = within_gross.melt(
    id_vars=['t', 's', 'r', 'Exports', 'exporter'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='importer',
    value_name='member'
)
within_gross['importer'] = within_gross['importer'].str.split('_').str[-1]
within_gross.dropna(inplace=True)
within_gross.drop(['member'], axis=1, inplace=True)

within_gross = within_gross.groupby(['t', 'exporter', 'importer'])['Exports'].sum().reset_index()
within_gross = within_gross[within_gross['exporter'] == within_gross['importer']]
within_gross.rename(columns={'exporter': 'rta', 'Exports': 'within'}, inplace=True)
within_gross.drop(['importer'], axis=1, inplace=True)

# End to end

within_flows = flows.merge(rta, left_on='s', right_on=f'{index}').drop([f'{index}'], axis=1)
within_flows = within_flows.melt(
    id_vars=['t', 's', 'r', 'flows'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='exporter',
    value_name='member'
)
within_flows['exporter'] = within_flows['exporter'].str.split('_').str[-1]
within_flows.dropna(inplace=True)
within_flows.drop(['member'], axis=1, inplace=True)

within_flows = within_flows.merge(rta, left_on='r', right_on=f'{index}').drop([f'{index}'], axis=1)
within_flows = within_flows.melt(
    id_vars=['t', 's', 'r', 'flows', 'exporter'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='importer',
    value_name='member'
)
within_flows['importer'] = within_flows['importer'].str.split('_').str[-1]
within_flows.dropna(inplace=True)
within_flows.drop(['member'], axis=1, inplace=True)

within_flows = within_flows.groupby(['t', 'exporter', 'importer'])['flows'].sum().reset_index()
within_flows = within_flows[within_flows['exporter'] == within_flows['importer']]
within_flows.rename(columns={'exporter': 'rta', 'flows': 'within'}, inplace=True)
within_flows.drop(['importer'], axis=1, inplace=True)

In [61]:
rci_gross = pd.merge(within_gross, exports_gross).merge(imports_gross).merge(world_gross)
rci_gross.insert(0, 'method', 'gross exports')

rci_flows = pd.merge(within_flows, exports_flows).merge(imports_flows).merge(world_flows)
rci_flows.insert(0, 'method', 'end-to-end')

rci = pd.concat([rci_gross, rci_flows]).reset_index(drop=True)
rci['share_within'] = (2 * rci['within']) / (rci['exports'] + rci['imports'])
rci['share_world'] = (rci['exports'] + rci['imports']) / (2 * rci['world'])
rci['rci'] = rci['share_within'] / rci['share_world']

rci.to_parquet(f'../data/{output2}.parquet', index=False)

In [62]:
rci

,method,t,rta,within,exports,imports,world,share_within,share_world,rci
0,gross exports,2017,asean,2.365401e+05,1.395722e+06,1.227044e+06,2.302460e+07,0.180375,0.056956,3.166924
1,gross exports,2017,eaeu,2.613153e+04,4.720038e+05,3.781996e+05,2.302460e+07,0.061471,0.018463,3.329440
2,gross exports,2017,eu,3.454208e+06,6.971020e+06,6.314668e+06,2.302460e+07,0.519989,0.288511,1.802323
3,gross exports,2017,nafta,1.112846e+06,3.333669e+06,3.892877e+06,2.302460e+07,0.307988,0.156931,1.962571
4,gross exports,2017,safta,2.483726e+04,4.695781e+05,5.316939e+05,2.302460e+07,0.049611,0.021744,2.281663
5,gross exports,2018,asean,2.654963e+05,1.623874e+06,1.486084e+06,2.552304e+07,0.170739,0.060925,2.802475
6,gross exports,2018,eaeu,3.049709e+04,5.830581e+05,4.005839e+05,2.552304e+07,0.062009,0.019270,3.217930
7,gross exports,2018,eu,3.777202e+06,7.862780e+06,7.243652e+06,2.552304e+07,0.500079,0.295937,1.689813
8,gross exports,2018,nafta,1.287064e+06,3.565520e+06,4.232532e+06,2.552304e+07,0.330099,0.152765,2.160828
9,gross exports,2018,safta,3.498872e+04,6.433296e+05,8.190831e+05,2.552304e+07,0.047851,0.028649,1.670246
